In [154]:
X.shape

(4, 3)

In [177]:
a

array([ 0.9,  0. ,  0.5])

In [178]:
b

array([ 0.8,  0. , -0.5])

In [179]:
c = a * b

In [180]:
c

array([ 0.72,  0.  , -0.25])

In [157]:
X.dot(syn0).shape

(4, 8)

In [ ]:
(4x3).dot(3x8)

In [ ]:
(4x8) kernels .... each of which would do 3 operations

In [181]:
import numpy as np

# Matrix Multiplication

X = np.array([ [0,0,1],[0,1,1],[1,0,1],[1,1,1] ])
syn0 = 2*np.random.random((3,2)) - 1

output = np.zeros((4,2))
for i in range(0,2):
    for j in range(0,4):
        output[j][i] = sum(X[j] * syn0[:,i])
        
output

array([[-0.73915087,  0.82002616],
       [-0.69207683,  1.32975542],
       [-1.02708853,  1.09020656],
       [-0.98001449,  1.59993582]])

In [146]:
a = np.array([0.9,0,0.5])
b = np.array([0.8,0,-0.5])

In [147]:
a

array([ 0.9,  0. ,  0.5])

In [148]:
b

array([ 0.8,  0. , -0.5])

In [149]:
a * b

array([ 0.72,  0.  , -0.25])

In [150]:
sum(a * b)

0.47000000000000008

In [97]:
X[2]

array([1, 0, 1])

In [98]:
syn0[:,1]

array([-0.98355323, -0.77255267, -0.26508825])

In [96]:
X[2] * syn0[:,1]

array([-0.98355323, -0.        , -0.26508825])

In [90]:
output[1][2] = sum(syn0[:,1]*X[2])

In [91]:
output[1][2]

-1.2486414819933214